In [1]:
# # Mount to Google Drive to train on Colab
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'/"SP22_ML_final_project"/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SP22_ML_final_project


In [2]:
import os

print(os.getcwd())

/content/drive/My Drive/SP22_ML_final_project


In [3]:
# Import Modules
import os

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from nets.yolo import YoloBody
from nets.yolo_training import (ModelEMA, YOLOLoss, get_lr_scheduler,
                                set_optimizer_lr, weights_init)
from utils.callbacks import LossHistory
from utils.dataloader import YoloDataset, yolo_dataset_collate
from utils.utils import get_classes
from utils.utils_fit import fit_one_epoch

In [4]:
# See the version
print("torch version:", torch.__version__)

torch version: 1.10.0+cu111


In [5]:
# Set Global Variables for traning
Cuda        = True if torch.cuda.is_available() else False
sync_bn     = False
fp16        = False # or "True"

classes_path = "model_data/facemask_classes.txt"
phi          = 's'  # yolo model size version
model_path   = 'model_data/yolox_%s.pth' % phi
input_shape  = [640, 640]
mosaic       = True

Init_Epoch              = 0
Freeze_Epoch            = 20
Freeze_batch_size       = 16
UnFreeze_Epoch          = 60
Unfreeze_batch_size     = 8
Freeze_Train            = True
Init_lr                 = 1e-2
Min_lr                  = Init_lr * 0.01
optimizer_type          = "adam"
momentum                = 0.937
weight_decay            = 5e-4
lr_decay_type           = "cos" # or "step"
save_period             = 1
save_dir                = 'logs'
num_workers             = 2
train_annotation_path   = './data/2007_train.txt'
val_annotation_path     = './data/2007_val.txt'

ngpus_per_node  = torch.cuda.device_count()
device          = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
local_rank      = 0
rank            = 0

In [6]:
# Set Variables for Model and Dataset
class_names, num_classes = get_classes(classes_path)
print(class_names)

model = YoloBody(num_classes, phi)
weights_init(model)
model_dict = model.state_dict()
pretrained_dict = torch.load(model_path, map_location = device)
pretrained_dict = {k: v for k, v in pretrained_dict.items() if np.shape(model_dict[k]) == np.shape(v)}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

yolo_loss = YOLOLoss(num_classes, fp16)
loss_history = LossHistory(save_dir, model, input_shape)

if fp16:
    from torch.cuda.amp import GradScaler as GradScaler
    scaler = GradScaler
else:
    scaler = None

model_train = model.train()
model_train = torch.nn.DataParallel(model)
cudnn.benchmark = True
model_train = model_train.cuda()
ema = ModelEMA(model_train) if rank == 0 else None

with open(train_annotation_path, encoding='utf-8') as f:
    train_lines = f.readlines()
with open(val_annotation_path, encoding='utf-8') as f:
    val_lines   = f.readlines()
num_train   = len(train_lines)
num_val     = len(val_lines)

['without_mask', 'with_mask', 'mask_weared_incorrect']
initialize network with normal type


/usr/local/lib/python3.7/dist-packages/torch/jit/_trace.py:965: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  argument_names,


In [7]:
# Training Process
UnFreeze_flag = False
if Freeze_Train:
    for param in model.backbone.parameters():
        param.requires_grad = False
batch_size = Freeze_batch_size if Freeze_Train else Unfreeze_batch_size

## Auto adjust leraing rate accroading to the batch_size
nbs             = 64
lr_limit_max    = 1e-3 if optimizer_type == 'adam' else 5e-2
lr_limit_min    = 3e-4 if optimizer_type == 'adam' else 5e-4
Init_lr_fit     = min(max(batch_size / nbs * Init_lr, lr_limit_min), lr_limit_max)
Min_lr_fit      = min(max(batch_size / nbs * Min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)

pg0, pg1, pg2 = [], [], []  
for k, v in model.named_modules():
    if hasattr(v, "bias") and isinstance(v.bias, nn.Parameter):
        pg2.append(v.bias)    
    if isinstance(v, nn.BatchNorm2d) or "bn" in k:
        pg0.append(v.weight)    
    elif hasattr(v, "weight") and isinstance(v.weight, nn.Parameter):
        pg1.append(v.weight)   
optimizer = {
    'adam'  : optim.Adam(pg0, Init_lr_fit, betas = (momentum, 0.999)),
    'sgd'   : optim.SGD(pg0, Init_lr_fit, momentum = momentum, nesterov=True)
}[optimizer_type]
optimizer.add_param_group({"params": pg1, "weight_decay": weight_decay})
optimizer.add_param_group({"params": pg2})

lr_scheduler_func = get_lr_scheduler(lr_decay_type, Init_lr_fit, Min_lr_fit, UnFreeze_Epoch)

epoch_step      = num_train // batch_size
epoch_step_val  = num_val // batch_size

if epoch_step == 0 or epoch_step_val == 0:
    raise ValueError("数据集过小，无法继续进行训练，请扩充数据集。")

ema.updates     = epoch_step * Init_Epoch

train_dataset   = YoloDataset(train_lines, input_shape, num_classes, epoch_length = UnFreeze_Epoch, mosaic=mosaic, train = True)
val_dataset     = YoloDataset(val_lines, input_shape, num_classes, epoch_length = UnFreeze_Epoch, mosaic=False, train = False)
train_sampler   = None
val_sampler     = None
shuffle         = True
gen             = DataLoader(train_dataset, shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True,
                                    drop_last=True, collate_fn=yolo_dataset_collate, sampler=train_sampler)
gen_val         = DataLoader(val_dataset  , shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True, 
                                    drop_last=True, collate_fn=yolo_dataset_collate, sampler=val_sampler)

In [8]:
for epoch in range(Init_Epoch, UnFreeze_Epoch):
    if epoch >= Freeze_Epoch and not UnFreeze_flag and Freeze_Train:
        batch_size = Unfreeze_batch_size

        nbs             = 64
        lr_limit_max    = 1e-3 if optimizer_type == 'adam' else 5e-2
        lr_limit_min    = 3e-4 if optimizer_type == 'adam' else 5e-4
        Init_lr_fit     = min(max(batch_size / nbs * Init_lr, lr_limit_min), lr_limit_max)
        Min_lr_fit      = min(max(batch_size / nbs * Min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)

        lr_scheduler_func = get_lr_scheduler(lr_decay_type, Init_lr_fit, Min_lr_fit, UnFreeze_Epoch)
        
        for param in model.backbone.parameters():
            param.requires_grad = True

        epoch_step      = num_train // batch_size
        epoch_step_val  = num_val // batch_size

        if epoch_step == 0 or epoch_step_val == 0:
            raise ValueError("数据集过小，无法继续进行训练，请扩充数据集。")
            
        ema.updates     = epoch_step * epoch
            
        gen             = DataLoader(train_dataset, shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True,
                                    drop_last=True, collate_fn=yolo_dataset_collate, sampler=train_sampler)
        gen_val         = DataLoader(val_dataset  , shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True, 
                                    drop_last=True, collate_fn=yolo_dataset_collate, sampler=val_sampler)

        UnFreeze_flag = True

    gen.dataset.epoch_now       = epoch
    gen_val.dataset.epoch_now   = epoch

    set_optimizer_lr(optimizer, lr_scheduler_func, epoch)

    fit_one_epoch(model_train, model, ema, yolo_loss, loss_history, optimizer, epoch, epoch_step, epoch_step_val, gen, gen_val, UnFreeze_Epoch, Cuda, fp16, scaler, save_period, save_dir, local_rank)
        
if local_rank == 0:
    loss_history.writer.close()

Start Train


Epoch 1/60:   0%|          | 0/43 [00:00<?, ?it/s<class 'dict'>]/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Epoch 1/60: 100%|██████████| 43/43 [01:37<00:00,  2.28s/it, loss=7.44, lr=0.0001]


Finish Train
Start Validation


Epoch 1/60: 100%|██████████| 4/4 [00:24<00:00,  6.02s/it, val_loss=5.45]


Finish Validation
Epoch:1/60
Total Loss: 7.436 || Val Loss: 5.454 
Start Train


...

Finish Validation
Epoch:59/60
Total Loss: 2.689 || Val Loss: 2.607 
Start Train


Epoch 60/60: 100%|██████████| 86/86 [01:03<00:00,  1.34it/s, loss=2.65, lr=1e-5]


Finish Train
Start Validation


Epoch 60/60: 100%|██████████| 9/9 [00:03<00:00,  2.29it/s, val_loss=2.64]


Finish Validation
Epoch:60/60
Total Loss: 2.650 || Val Loss: 2.642 
